In [5]:
import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_gradient
from fmm.surface import scale_surface

# Adaptoctree
import adaptoctree.morton as morton
import adaptoctree.tree as tree

In [3]:
# ! fmm generate-test-data -c test_random && fmm compute-operators -c test_random

In [6]:
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)
laplace_gradient(e.sources, e.targets, e.source_densities)

array([[ 15.34029  ,   2.0270848,  71.88409  ],
       [ 52.69213  ,  58.177074 ,  38.692673 ],
       [ 86.45617  ,  23.862509 ,  68.68593  ],
       ...,
       [-31.813215 , -64.628845 , -57.76195  ],
       [-49.487106 , -21.039127 , -26.436289 ],
       [-49.536842 ,   4.2401443,  -0.6216814]], dtype=float32)

In [7]:
e = Fmm('test_random')

In [8]:
e.nleaves

32768

In [9]:
# direct = laplace_p2p_serial(
#     e.sources,
#     e.targets,
#     e.source_densities
# )

In [62]:
! rm C4E4.hdf5 && fmm generate-test-data -c C4E4 && fmm compute-operators -c C4E4
! rm C5E5.hdf5 && fmm generate-test-data -c C5E5 && fmm compute-operators -c C5E5
! rm C6E6.hdf5 && fmm generate-test-data -c C6E6 && fmm compute-operators -c C6E6
! rm C7E7.hdf5 && fmm generate-test-data -c C7E7 && fmm compute-operators -c C7E7

Generating sphere sources & targets
Computing operators
Computing octree
^C
Generating sphere sources & targets
Computing operators
Computing octree
Computing Inner Surface of Order 5
Computing Outer Surface of Order 5
Computing Inverse of Check To Equivalent Gram Matrix
Computing M2M & L2L Operators
Computed (1/8) M2M/L2L operators
Computed (2/8) M2M/L2L operators
Computed (3/8) M2M/L2L operators
Computed (4/8) M2M/L2L operators
Computed (5/8) M2M/L2L operators
Computed (6/8) M2M/L2L operators
Computed (7/8) M2M/L2L operators
Computed (8/8) M2M/L2L operators
Computed operators for (1/5) M2L Levels
Computed operators for (2/5) M2L Levels
^C
Traceback (most recent call last):
  File "/home/sri/Github/pyexafmm/scripts/precompute_operators.py", line 635, in <module>
    main(**config)
  File "/home/sri/Github/pyexafmm/scripts/precompute_operators.py", line 617, in main
    compute_m2l(
  File "/home/sri/Github/pyexafmm/scripts/precompute_operators.py", line 567, in compute_m2l
    u, s, v

In [22]:
# Discretisation order of check surface
Cvec = [4, 5, 6, 7]

# Discretisation order of equivalent surface
Evec = [4, 5, 6, 7]

# Load experimental data into PyExaFMM
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

In [31]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'C{p}E{p}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

In [ ]:
# Evaluate PyExaFMM experiments
for e in pyfmmvec:
    e.run()
    

In [32]:
# Evaluate ExaFMM-T experiments
trg_values = []
for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    trg_values.append(laplace.evaluate(tree, fmm))

In [30]:
direct

array([48807.414, 50649.754, 49043.723, ..., 48949.71 , 48578.098,
       49259.96 ], dtype=float32)

In [59]:
pyerrvec = []
exerrvec = []

for i in range(len(pyfmmvec)):
    pyerrvec.append(np.mean(abs(pyfmmvec[i].target_potentials[:,0]-direct)/direct))
    exerrvec.append(np.mean(abs(trg_values[i][:,0]-direct)/direct))

In [60]:
pyerrvec

[8.759181e-05, 2.8106256e-06, 1.5909669e-06, 1.7051891e-06]

In [61]:
exerrvec

[7.663393e-05, 3.9089655e-06, 3.4128896e-06, 3.843031e-06]

In [56]:
pyfmmvec[-1].target_potentials

array([[ 48807.277,  39147.848,  41694.375,  43357.35 ],
       [ 50649.586,  48898.51 ,  48978.066,  35368.6  ],
       [ 49043.566,  44093.234,  41412.508,  40503.21 ],
       ...,
       [ 48949.668, -41548.613, -39886.14 , -43950.746],
       [ 48578.094, -38297.74 , -41545.324, -43492.91 ],
       [ 49259.82 , -39362.04 , -42010.234, -48220.566]], dtype=float32)

In [57]:
direct

array([48807.414, 50649.754, 49043.723, ..., 48949.71 , 48578.098,
       49259.96 ], dtype=float32)

In [48]:
pyfmmvec[0].target_potentials[:,0]

array([48811.742, 50656.63 , 49047.523, ..., 48953.797, 48582.863,
       49263.883], dtype=float32)

In [35]:
pyfmmvec[0].target_potentials[:,0]

array([48811.742, 50656.63 , 49047.523, ..., 48953.797, 48582.863,
       49263.883], dtype=float32)

In [39]:
trg_values[0][:,0]

array([48811.336, 50654.934, 49047.098, ..., 48953.33 , 48582.316,
       49263.02 ], dtype=float32)